In [ ]:
#default_exp utils.tmy

In [ ]:
#export
from capetools.imports import *
import dateutil
import pandas as pd
import pytz

# TMY: Typical Meteorological Year
> Reading different types of tmy files

In [ ]:
DATAPATH = Path.cwd().parent/'data'
DATAPATH.ls()

(#2) [Path('/home/tc256760/Documents/capetools/data/pvgis_tmy_chambery.csv'),Path('/home/tc256760/Documents/capetools/data/sample_data.hdf')]

## [PVGIS](https://re.jrc.ec.europa.eu/pvgis_tools)
One can download TMY files for EU from this website
- The files structure changed recently (Nov 2019) so this function should work.


In [ ]:
#export
def read_pvgis(filename: File, coerce_year: int=None, rename_cols: bool=True):
    "Utility function to read PVGIS tmy files"
    with open(filename, 'r') as csvdata:
        # read in file metadata, advance buffer to second line
        gps_data = {}
        for _ in range(3):
            line = csvdata.readline()
            attribute = line.rstrip('\n').split(":")
            gps_data[attribute[0][0:-1]] = float(attribute[1])

        months_year = pd.read_csv(csvdata, nrows=12, engine='python').set_index('month')
    
        def _parsedate(date_str, year=None):
            "parser to get a coerced year from a tmy file"
            date = pd.datetime.strptime(date_str, '%Y%m%d:%H%M')
            if year is not None:
                date = date.replace(year=year)
            return date


        col_names = 'time(UTC),T2m,RH,G(h),Gb(n),Gd(h),IR(h),WS10m,WD10m,SP'.split(',')
        new_names = 'date,temp,humidity,ghi,dni,dhi,infra,ws,wd,pressure'.split(',')
        date_col = col_names[0]
        data = pd.read_csv(csvdata,
                           header=0,
                           names=col_names,
                           parse_dates=True,
                           date_parser=lambda x: _parsedate(x, year=coerce_year),
                           index_col=date_col,
                           nrows=365*24)
        return gps_data,months_year,data.rename(columns=dict(zip(col_names, new_names)))


We can ingest a tmy from pvgis and coerce on year 2019 using this function:

In [ ]:
gps_data, months_year, data = read_pvgis(DATAPATH/'pvgis_tmy_chambery.csv', coerce_year=2019)

In [ ]:
gps_data

{'Latitude (decimal degrees': 45.641,
 'Longitude (decimal degrees': 5.875,
 'Elevation (m': 235.0}

We also recover the correspondance

In [ ]:
months_year

,year
month,
1,2012
2,2007
3,2007
4,2016
5,2012
6,2007
7,2008
8,2014
9,2016


In [ ]:
data.head()

,temp,humidity,ghi,dni,dhi,infra,ws,wd,pressure
time(UTC),,,,,,,,,
2019-01-01 00:00:00,2.87,88.28,0.0,-0.0,0.0,259.63,1.33,170.0,99504.0
2019-01-01 01:00:00,3.59,90.07,0.0,-0.0,0.0,268.30,1.39,166.0,99508.0
2019-01-01 02:00:00,4.32,91.86,0.0,-0.0,0.0,276.97,1.45,162.0,99511.0
2019-01-01 03:00:00,5.04,93.64,0.0,-0.0,0.0,285.64,1.51,167.0,99517.0
2019-01-01 04:00:00,5.76,95.43,0.0,-0.0,0.0,294.32,1.57,171.0,99524.0


## Other TMY reading method
> May be useful, don't remeber.

In [ ]:
#export
def read_tmy(filename: File, columns:Optional[list]=None, coerce_year:Optional[int]=None):
    "Reads TMY files with dates on 5 first cols."
    if columns is None:
        columns = ['ghi', 'dni', 'dhi','temp_air', 'temp_dew', 
                   'humidity', 'air_pressure','wind_speed', 'wind_dir', 'snow']
    with open(filename, 'r') as csvdata:
        # read in file metadata, advance buffer to second line
        firstline = csvdata.readline().rstrip('\n').split(",")
        secondline = csvdata.readline().rstrip('\n').split(",")
        meta = dict(zip(firstline,secondline))
        meta['Latitude'] = float(meta['Latitude'])
        meta['Longitude'] = float(meta['Longitude'])
        meta['Elevation'] = float(meta['Elevation'])
        meta['Time Zone'] = int(meta['Time Zone'])
        fixed_tz = pytz.FixedOffset(float(meta['Time Zone']) * 60)
        def _parsedate(ymdh, year=None):
            "parser to get a coerced year from a tmy file"
            ymdh = pd.datetime.strptime(ymdh, '%Y %m %d %H %M')
            if year is not None:
                ymdh = ymdh.replace(year=year)
            return ymdh  
        data = pd.read_csv(csvdata, header=0, 
                           index_col='datetime', 
                           parse_dates={'datetime': [0,1,2,3,4]}, 
                           date_parser=lambda x: _parsedate(x, year=coerce_year)
                          )
        data.columns = columns
        return meta, data.tz_localize(fixed_tz)

# Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.core.ipynb.
Converted 01_utils.pandas_hacks.ipynb.
Converted 03_utils.matlab_io.ipynb.
Converted 03_utils.tmy.ipynb.
Converted 04_utils.missing.ipynb.
Converted 05_utils.plot.ipynb.
Converted 06_modelling.mybifacialvf.ipynb.
Converted 07_modelling.mypvfactors.ipynb.
Converted index.ipynb.
